In [2]:
from open_crypto import runner
import sqlite3 
import pandas as pd
import datetime


#create database
depth_db = pd.DataFrame(columns=['exchange', 'mid price', 'spread', 'depth 50 bps','depth 50 USD', 'depth 100 bps','depth 100 USD', 'depth 150 bps','depth 150 USD','depth 200 bps','depth 200 USD', 'time'])

In [3]:
def calculate_depth(database, exchange, range): #depths are calculated in BPS
    range = range / 100 #convert to kBPS (1000BPS = 1KBPS)
    #get midprice of the last range of trades
    price = prices.loc[prices['exchange'] == exchange, 'price'].item()

    #calculate depth boundaries
    depth_asks = price * (1 + range)
    depth_bids = price - (price * range)

    #get trades within the depth boundaries
    trades = database.loc[(database['bids_price'] > depth_bids) & (database['asks_price'] < depth_asks)]
    trades = trades.assign(**{'price': trades['bids_price'].fillna(trades['asks_price'])})
    

    #calculate depth
    depth = trades['price'].std()

    return depth

def calculate_spread(database, exchange):
    #get midprice of the last range of trades
    bidmaxprice = database.loc[database['exchange'] == exchange, 'bids_price'].max()
    askminprice = database.loc[database['exchange'] == exchange, 'asks_price'].min()

    #calculate spread
    spread = bidmaxprice - askminprice

    return spread

def fetch_data():
    for exchange in orderbook['exchange'].unique():
        depth_val_50 = calculate_depth(orderbook, exchange, 50)
        depth_val_100 = calculate_depth(orderbook, exchange, 100)
        depth_val_150 = calculate_depth(orderbook, exchange, 150)
        depth_val_200 = calculate_depth(orderbook, exchange, 200)
        spread = calculate_spread(orderbook, exchange)
        depth_val_50_USD = depth_val_50 * prices.loc[prices['exchange'] == exchange, 'price'].item()
        depth_val_100_USD = depth_val_100 * prices.loc[prices['exchange'] == exchange, 'price'].item()
        depth_val_150_USD = depth_val_150 * prices.loc[prices['exchange'] == exchange, 'price'].item()
        depth_val_200_USD = depth_val_200 * prices.loc[prices['exchange'] == exchange, 'price'].item()
        row = [exchange, prices.loc[prices['exchange'] == exchange, 'price'].item(), spread, depth_val_50, depth_val_50_USD, depth_val_100, depth_val_100_USD, depth_val_150, depth_val_150_USD, depth_val_200, depth_val_200_USD, cur_time] 
        depth_db.loc[len(depth_db)] = row

def check_dbstatus():
    if con.total_changes == 0:
        return(print("DB connection established"))
    else:
        return(print("failed to establish connection to database"))



In [4]:
#fetch data from exchanges
#fetch data from exchanges
runner.run('amkt.yaml')
# Read sqlite query results into a pandas DataFrame

con = sqlite3.connect("amkt_data.db")

check_dbstatus()

orderbook = pd.read_sql_query('SELECT * FROM order_books_view;', con)
prices = pd.read_sql_query('SELECT exchange, MAX(price) AS price FROM trades_view GROUP BY exchange;', con)
# Verify that result of SQL query is stored in the dataframe
#get the time of the snapshot
cur_time = datetime.datetime.now() 
fetch_data()

con.close()

Database amkt_data created.

Requesting 2 exchange(s) for job: orderbook.                                    
Requesting 2 exchange(s) for job: tickers.
Requesting data... 50.00 % /                                                    Pair-ID 12 - Coinbase: 1000 tuple(s)
Pair-ID 663 - Ftx: 100 tuple(s)
Terminating.
Pair-ID 12 - Coinbase: 15787 tuple(s)                                           
Pair-ID 663 - Ftx: 100 tuple(s)
Terminating.
DB connection established


In [5]:
depth_db

,exchange,mid price,spread,depth 50 bps,depth 50 USD,depth 100 bps,depth 100 USD,depth 150 bps,depth 150 USD,depth 200 bps,depth 200 USD,time
0,COINBASE,1736.71,-0.21,115.445531,200495.407881,212.620287,369259.778295,296.638821,515175.607671,472.546695,820676.570965,2022-07-29 13:52:40.950280
1,FTX,1728.40,-0.10,114.403061,197734.250175,210.620198,364035.949692,294.665322,509299.543125,469.908038,812189.053170,2022-07-29 13:52:40.950280
